# PROYECTO DE INTELIGENCIA ARTIFICIAL: Seleccion de características

Las siguientes funciones se encargan de obtener los atributos con los que trabajaremos, ademas de obtener los atributos ordenados y los objetivos para ambos csv.

Importaciones:

In [89]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

from sklearn.tree import DecisionTreeClassifier

from matplotlib import pyplot
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

In [6]:
def lectura_titanic():
    fichero = pd.read_csv('titanic.csv')    
    return fichero

def lectura_breastCancer():
    fichero = pd.read_csv('BreastCancer.csv')
    return fichero

In [7]:
titanic = lectura_titanic()
breast_cancer = lectura_breastCancer()

In [8]:
# Eleccion de semilla inicial aleatoria.
np.random.seed(357823)

# Atributos discretos, continuos y objetivo de titanic.csv.
atributos_discretos_titanic = ['Sex', 'Embarked', 'Alone', 'Deck']
atributos_continuos_titanic = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Initial', 
                           'Age_band', 'Family_Size', 'Fare_cat', 'Title', 'Is_Married']
atributos_titanic = titanic.loc[:, atributos_discretos_titanic + atributos_continuos_titanic]
objetivo_titanic = titanic['Survived']

# Atributos discretos, continuos y objetivo de BreastCancer.csv.
atributos_continuos_breast_cancer = ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 
                           'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 
                           'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error',
                           'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 
                           'worst perimeter', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points',
                           'worst symmetry', 'worst fractal dimension']
																				
atributos_breast_cancer = breast_cancer.loc[:, atributos_continuos_breast_cancer]
objetivo_breast_cancer = breast_cancer['diagnosis']

##### A continuación realizaremos los algoritmos de busqueda con los que vamos a trabajar. Para ello, trabajaremos con el algoritmo de "Decision tree clasifier" y el algoritmo de "Naives bayes".

#### Decision tree clasifier

In [124]:
def decision_tree(atributos, objetivo, atributos_discretos):
    codificador_atributos_discretos = OrdinalEncoder()
    codificador_atributos_discretos.fit(atributos[atributos_discretos])
    
    atributos[atributos_discretos] = codificador_atributos_discretos.transform(atributos[atributos_discretos])

    codificador_objetivo = LabelEncoder()
    objetivo = codificador_objetivo.fit_transform(objetivo)

    clasificador_CART = DecisionTreeClassifier()
    clasificador_CART.fit(atributos, objetivo)
    print()
    print('Número de atributos detectados:',
      f'{codificador_atributos_discretos.n_features_in_}')
    print()
    print('Nombres de los atributos detectados:')
    print(f'{codificador_atributos_discretos.feature_names_in_}')
    print()
    print('Categorías detectadas de cada atributo:')
    for atributo, categorías in zip(
        codificador_atributos_discretos.feature_names_in_,
        codificador_atributos_discretos.categories_):
        print(f'{atributo}: {categorías}')
    print()
    print(f'Clases detectadas: {codificador_objetivo.classes_}')
    print()
    return clasificador_CART

In [125]:
# DECISION TREE CLASIFIER
decision_tree(atributos_titanic, objetivo_titanic, atributos_discretos_titanic)


Número de atributos detectados: 4

Nombres de los atributos detectados:
['Sex' 'Embarked' 'Alone' 'Deck']

Categorías detectadas de cada atributo:
Sex: [0. 1.]
Embarked: [0. 1. 2.]
Alone: [0. 1.]
Deck: [0. 1. 2. 3. 4. 5. 6. 7. 8.]

Clases detectadas: [0 1]



DecisionTreeClassifier()

#### Funcion de rendimiento

In [128]:
def rendimiento(atributos, objetivo, feature, atributos_discretos):
    (atributos_entrenamiento, atributos_prueba,
     objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)
    tree = decision_tree(atributos, objetivo, atributos_discretos)
    score = cross_validate(tree, atributos_entrenamiento[feature], objetivo_entrenamiento, cv = 7, scoring = 'balanced_accuracy')

    print(f'Cantidad inicial de ejemplos: {objetivo.size}',
      f'(80 % = {objetivo.size * .8}, 20 % = {objetivo.size * .2})')
    print('Atributos:',
      f'Entrenamiento ({atributos_entrenamiento.shape[0]} ejemplos)',
      f'Prueba ({atributos_prueba.shape[0]} ejemplos)')
    print('Objetivo:',
      f'Entrenamiento ({objetivo_entrenamiento.shape[0]} ejemplos)',
      f'Prueba ({objetivo_prueba.shape[0]} ejemplos)')
    print('')
    print('También podemos comprobar como se mantiene la proporción de ejemplos en cada clase tanto para el subconjunto de entrenamiento como para el de prueba.')
    print('Proporción inicial de ejemplos:')
    print(pd.Series(objetivo).value_counts(normalize=True))
    print('Proporción de ejemplos de entrenamiento:')
    print(pd.Series(objetivo_entrenamiento).value_counts(normalize=True))
    print('Proporción de ejemplos de prueba:')
    print(pd.Series(objetivo_prueba).value_counts(normalize=True))
    print('')
    return score

In [129]:
feature = ['Pclass', 'Age']
rendimiento(atributos_titanic, objetivo_titanic, feature, atributos_discretos_titanic)


Número de atributos detectados: 4

Nombres de los atributos detectados:
['Sex' 'Embarked' 'Alone' 'Deck']

Categorías detectadas de cada atributo:
Sex: [0. 1.]
Embarked: [0. 1. 2.]
Alone: [0. 1.]
Deck: [0. 1. 2. 3. 4. 5. 6. 7. 8.]

Clases detectadas: [0 1]

Cantidad inicial de ejemplos: 891 (80 % = 712.8000000000001, 20 % = 178.20000000000002)
Atributos: Entrenamiento (712 ejemplos) Prueba (179 ejemplos)
Objetivo: Entrenamiento (712 ejemplos) Prueba (179 ejemplos)

También podemos comprobar como se mantiene la proporción de ejemplos en cada clase tanto para el subconjunto de entrenamiento como para el de prueba.
Proporción inicial de ejemplos:
0    0.616162
1    0.383838
Name: proportion, dtype: float64
Proporción de ejemplos de entrenamiento:
0    0.616573
1    0.383427
Name: proportion, dtype: float64
Proporción de ejemplos de prueba:
0    0.614525
1    0.385475
Name: proportion, dtype: float64



{'fit_time': array([0.0019989 , 0.00200009, 0.00200057, 0.00199962, 0.00100064,
        0.00099969, 0.0009985 ]),
 'score_time': array([0.00100017, 0.00100017, 0.00099921, 0.00100017, 0.00100088,
        0.00099969, 0.00101542]),
 'test_score': array([0.62393162, 0.61782662, 0.61477411, 0.63553114, 0.70879121,
        0.68382961, 0.76406121])}